In [2]:
import numpy as np
import pandas as pd
import pdb
import setuptools
from multipledispatch import dispatch
# from multimethods import multimethod

%matplotlib inline 
%load_ext cython 

In [2]:
def signature(*types):
    def func(f):
        def inner_func(callingObj, *args, **kwargs):
            return f(callingObj, *args, **kwargs)
        inner_func.signature = types
        return inner_func
    return func
        
class Overload(object):
    def __init__(self, func):
        self.owner = None
        self.signatures = []
        self.methods = []
        self.methods.append(func)
        self.signatures.append(func.signature)
        
    def __get__(self, owner, ownerType=None):
        self.owner = owner or self
        return self
    
    def __call__(self, *args, **kwargs):
        signature = []
        for arg in args:
            signature.append(arg.__class__.__name__)
        for _, v in kwargs:# 此处需使用强制签名，否则会产生字典乱序
            signature.append(v.__class__.__name__)
        signature = tuple(signature)
        if signature in self.signatures:
            index = self.signatures.index(signature)
        else:
            raise Exception("There is no overload for this method with this signature.")
        return self.methods[index](self.owner, *args, **kwargs)
    
    def overload(self, func):
        self.methods.append(func)
        self.signatures.append(func.signature)
        return self

In [19]:
%%cython 
import numpy as np
cimport numpy as np
# from libcpp cimport bool
# from multimethods import multimethod
# Dtype_t = np.float32_t

# from multipledispatch import dispatch

# @multimethod(np.ndarray[np.float32_t])

ctypedef fused Dtype_t:
    np.float32_t
    np.float64_t
    np.int32_t

cdef hint_value_mean_tj_cy1(np.ndarray[Dtype_t] vec):
    res = 0
    m = vec.mean()
#     print m
    if vec[0] - m < 0:
        flag_small = True
    else:
        flag_small = False
    for v in vec[1:]:
        if v - m < 0:#现在在均线下方
            if flag_small :#之前都是在均线下方
                continue
            else:
#                 print v,flag_small
                res += 1
                flag_small = True
        else:#现在在均线上方
            if not flag_small:#之前都是在均线上方
                continue
            else:
#                 print v,flag_small
                res += 1
                flag_small = False
    return res

cdef int hint_value_mean_tj_cy2(np.ndarray[Dtype_t] vec):
    
    cdef int res = 0
    cdef double m = vec.mean()
    cdef double v 
    
#     cdef bool flag_small
#     print m
    if vec[0] - m < 0:
        flag_small = True
    else:
        flag_small = False
    for v in vec[1:]:
        if v - m < 0:#现在在均线下方
            if flag_small :#之前都是在均线下方
                continue
            else:
#                 print v,flag_small
                res += 1
                flag_small = True
        else:#现在在均线上方
            if not flag_small:#之前都是在均线上方
                continue
            else:
#                 print v,flag_small
                res += 1
                flag_small = False
    return res

print hint_value_mean_tj_cy1 [np.float32_t] (np.array([-0.47669166 , 0.44250423 , 1.0235505],dtype=np.float32))
print hint_value_mean_tj_cy2 [np.float32_t] (np.array([-0.47669166 , 0.44250423 , 1.0235505],dtype=np.float32))
print hint_value_mean_tj_cy2 [np.float64_t] (np.array([-0.47669166 , 0.44250423 , 1.0235505],dtype=np.float64))
print hint_value_mean_tj_cy2 [np.int32_t] (np.array([2 , 2 , 1],dtype=np.int32))

c:\users\administrator\anaconda2\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Administrator\.ipython\cython\_cython_magic_8186c0851c2d040405e736984d9fe39c.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


1
1
1
1
